In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [37]:
train = sns.load_dataset("titanic")
train.head(5)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [38]:
train["sex"] == "male"
train.loc[train["sex"] == "male", "sex_encode"] = 0
train.loc[train["sex"] == "female", "sex_encode"] = 1

In [39]:
train = train.dropna()

In [40]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 1 to 889
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     182 non-null    int64   
 1   pclass       182 non-null    int64   
 2   sex          182 non-null    object  
 3   age          182 non-null    float64 
 4   sibsp        182 non-null    int64   
 5   parch        182 non-null    int64   
 6   fare         182 non-null    float64 
 7   embarked     182 non-null    object  
 8   class        182 non-null    category
 9   who          182 non-null    object  
 10  adult_male   182 non-null    bool    
 11  deck         182 non-null    category
 12  embark_town  182 non-null    object  
 13  alive        182 non-null    object  
 14  alone        182 non-null    bool    
 15  sex_encode   182 non-null    float64 
dtypes: bool(2), category(2), float64(3), int64(4), object(5)
memory usage: 19.7+ KB


In [41]:
# class열은 종속변수(y), 나머지는 독립변수(X)
data = train[['pclass', 'sex_encode', 'age', 'fare']]
target = train['survived']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.2, random_state=42)

### cross_validate() 함수로 교차 검증 확인

In [42]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, X_train, y_train, return_train_score=True, n_jobs=-1)

np.mean(scores['train_score']), np.mean(scores['test_score'])

(1.0, 0.7310344827586206)

### 랜덤 포레스트의 매개변수

In [43]:
# 랜덤 포레스트 모델을 훈련세트로 훈련한 후 특성 중요도 확인
rf.fit(X_train, y_train)
rf.feature_importances_

array([0.01886937, 0.30854526, 0.33817579, 0.33440959])

### 랜덤 포레스트의 자체 모델 평가 점수

In [44]:
# 새로운 랜덤포레스트 객체를 이용하여 oob_score 확인
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(X_train, y_train)
rf.oob_score_

0.7448275862068966

## 엑스트라 트리(Extra Trees)

In [45]:
# ExtraTreesClassifier를 적용하여 교차 검증 점수 확인
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, X_train, y_train, return_train_score=True, n_jobs=-1)

np.mean(scores['train_score']), np.mean(scores['test_score'])

(1.0, 0.7241379310344827)

In [47]:
# 특성 중요도 확인
et.fit(X_train, y_train)
et.feature_importances_
# pclass, sex_encode, age, fare

array([0.02296949, 0.33551284, 0.34219411, 0.29932357])

## 그레이디언트 부스팅(Gradient boosting)

In [48]:
# GradientBoostingClassifier 사용
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, X_train, y_train, return_train_score=True, n_jobs=-1)

np.mean(scores['train_score']), np.mean(scores['test_score'])

(1.0, 0.7517241379310345)

In [49]:
# 트리 개수와 학습률을 조정하여 학습
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, X_train, y_train, return_train_score=True, n_jobs=-1)

np.mean(scores['train_score']), np.mean(scores['test_score'])

(1.0, 0.7172413793103447)

In [50]:
# 특성 중요도 확인
gb.fit(X_train, y_train)
gb.feature_importances_

array([0.00389828, 0.39447195, 0.35691544, 0.24471434])